In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/logistics-shopee-code-league/delivery_orders_march.csv')
len(df)

In [ ]:
df.head(10)

In [ ]:
'''
def get_address(data):
    buyers = []
    sellers = []
    for i in range(len(data)):
        buyers.append(data['buyeraddress'][i].split(" ")[-1].lower())
        sellers.append(data['selleraddress'][i].split(" ")[-1].lower())
    return buyers, sellers
'''

In [ ]:
time_limit = [[3,5,7,7],[5,5,7,7],[7,7,7,7],[7,7,7,7]]

time_limits = pd.DataFrame(time_limit, columns=['manila','luzon','visayas','mindanao'], index=['manila','luzon','visayas','mindanao'])
time_limits

In [ ]:
def get_time(data):
    return datetime.datetime.fromtimestamp(data).strftime('%Y-%m-%d %H:%M:%S').split(" ")[0]

def get_epoch(data):
    return datetime.datetime(data).timestamp()

def timestamp(data):
    return time.mktime(datetime.datetime.strptime(data, "%Y-%m-%d").timetuple())

holidays = ['2020-03-01','2020-03-08','2020-03-15','2020-03-22', '2020-03-25', '2020-03-29', '2020-03-30', '2020-03-31']
holi_epoch = []

for i in range(len(holidays)):
    holi_epoch.append(timestamp(holidays[i]))
    
holi_epoch

In [ ]:
timestamp(get_time(df['pick'][4139])) - timestamp(get_time(df['pick'][0]))

In [ ]:
def extra_days(pick, arrival, holi):
    addition = 0
    for i in range(len(holi)):
        if (arrival > holi[i]) & (pick < holi[i]):
            addition += 1
        else:
            break
    return addition

def get_deadline(buyer, seller, limit):
    return limit[buyer][seller]

'''
def first_del(pick, first, buyer, seller, holidays, limit):
    deadline = get_deadline(buyer, seller, limit)
    deadline += extra_days(first, holidays)
    deadline *= 86400
    interval = first - pick
    return (interval < deadline)
'''

def first_del(data, holi, limit):
    deadline = get_deadline(data['buyeraddress'].split(" ")[-1].lower(), data['selleraddress'].split(" ")[-1].lower(), limit)
    deadline += extra_days(data['pick'], data['1st_deliver_attempt'], holi)
    deadline *= 86400
    interval = data['1st_deliver_attempt'] - data['pick']
    return (interval < deadline)
    
def sec_del(data, holi):
    deadline = 3
    deadline += extra_days(data['pick'], data['2nd_deliver_attempt'], holi)
    deadline *= 86400
    interval = data['2nd_deliver_attempt'] - data['1st_deliver_attempt']
    return (interval < deadline)

In [ ]:
def get_late(data, holi, limit):
    orderid = []
    is_late = []
    for i in range(len(data)):
        if data['2nd_deliver_attempt'].isnull()[i] == True:
            check1 = first_del(data.iloc[i], holi, limit)
            check2 = True
        else:
            check1 = first_del(data.iloc[i], holi, limit)
            check2 = sec_del(data.iloc[i], holi)
        check = check1 & check2
        if check == False:
            check3 = 0
        else:
            check3 = 1
        orderid.append(data['orderid'][i])
        is_late.append(check3)
        print(i)
    return orderid, is_late

orderid, is_late = get_late(df, holi_epoch, time_limits)

In [ ]:
result = pd.DataFrame(is_late)
result.to_csv('unfinished.csv', index=False)

In [ ]:
result = pd.DataFrame([orderid, is_late])
result.to_csv('attempt1.csv', index=False)